<a href="https://colab.research.google.com/github/schalckens/tp-information-retrieval-with-llm-student-version/blob/main/1-Recherche%20d'information%20classique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Partie 1. - Recherche d'Information classique

Dans cette partie, nous allons mettre en oeuvre des principes et modèles classiques de Recherche d'Information. Le jeu de donnée est une collection de livres au format texte (.txt) de Henry Rider Haggard. Jetez un oeil à ces documents dans le dossier _data_.

En sortie de ce module, vous serez capable de :

- Construire un index inversé ;
- Effectuer des requêtes simples selon le modèle booléen :
- Calculer la pondération des termes selon la méthode TF-IDF ;
- Mettre en oeuvre le modèle vectoriel de recherche d'information et y appliquer des requêtes.

### Import des bibliothèques logicielles et configuration

Les lignes suivantes permettent d'instancier un objet la classe `IRSystem` représentant notre moteur de recherche et de charger les données en RAM.

In [ ]:
import os

# Vérifie si le code est exécuté sur Google Colab
if 'COLAB_GPU' in os.environ:
    # Commandes à exécuter uniquement sur Google Colab
    !git clone https://github.com/schalckens/tp-information-retrieval-with-llm-student-version.git
    %cd tp-information-retrieval-with-llm-student-version
else:
    # Commandes à exécuter si ce n'est pas sur Google Colab
    print("Pas sur Google Colab, ces commandes ne seront pas exécutées.")

fatal: destination path 'tp-information-retrieval-with-llm-student-version' already exists and is not an empty directory.
/content/tp-information-retrieval-with-llm-student-version


#### Chargement des données

Les lignes ci-dessous permettent de charger les données qui sont un ensemble de 60 livres au format texte (.txt) d'[Henry Rider Haggard ](https://fr.wikipedia.org/wiki/Henry_Rider_Haggard).


In [ ]:
from classic_ir.IRSystem import *

# !rm -rf ./data/RiderHaggard/stemmed
ir_system = IRSystem()
ir_system.read_data('./data/RiderHaggard') # chargement des données et prétraitement des documents (stemming).

Reading in documents...
Already stemmed!


### Exercice 1. - Construction de l'index inversé

Ce premier exercice a pour objectif de construire l'index inversé non positionnel. L'attribut `self.inverted_index` est un tableau associatif qui associe une liste d'entiers (docIDs) à un mot (word).

Documentation ici https://docs.python.org/3/library/collections.html#collections.defaultdict.

Exercice : modifier la fonction `index` pour calculer l'index inversé.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Inverted Index Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [ ]:
# Exercice 1. Indexation

def index(self):
    """
    Construit l'index inversé et le stocke dans self.inverted_index.

    Dans le code ci-dessous, seul le dictionnaire des tokens est construit. Les postings lists sont vides.
    """
    print("Indexing...")
    self.tf = defaultdict(Counter) # tf est un dictionnaire qui à un document associe un Counter de mots.
    inverted_index = defaultdict(list) # inverted_index est un dictionnaire qui à un mot associe une liste de documents.
    i=0
    for doc in self.docs :
      for word in doc :
        if(i not in inverted_index[word] ) :
          inverted_index[word].append(i)
      i=i+1

    self.inverted_index = inverted_index

# Ne pas modifier les lignes ci-dessous
IRSystem.index = index
ir_system.index()
run_tests(ir_system, part=0)

Indexing...
defaultdict(<class 'list'>, {'titl': [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 25, 26, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59], 'a': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'winter': [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 17, 18, 19, 20, 22, 23, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59], 'pilgrimag': [0, 6, 12, 29, 31, 34, 36, 37, 46, 49], 'author': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 'h': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9

### Exercice 2. - Recherche booléenne

Ce deuxième exercice a pour objectif d'implémenter la recherche booléenne. La requête `query` est une liste de mots _lemmatisés_ et l'algorithme doit rechercher les documents qui contiennent TOUS ces mots.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.


Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Boolean Retrieval Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [ ]:
# Exercice 2. Recherche booléenne
def boolean_retrieve(self, query):
    """
    A partir d'une requête sous la forme d'une liste de mots *lemmatisés*,
    retourne la liste des documents dans lesquels *tous* ces mots apparaissent (ie une requête AND).
    Retourne une liste vide si la requête ne retourne aucun document.

    Dans le code ci-dessous, tous les documents répondent.
    """
    print(query)
    docs = list()
    boll = True
    i=0
    for doc in self.docs:
      for word in query:
        if (word not in doc):
          boll = False
      if(boll):
        docs.append(i)
      i+=1
      boll=True
    return docs

# Ne pas modifier les lignes ci-dessous
IRSystem.boolean_retrieve = boolean_retrieve
run_tests(ir_system, part=1)

===== Running tests =====
Boolean Retrieval Test
['separ', 'of', 'church', 'and', 'state']
['priestess', 'ritual', 'sacrific']
['demon', 'versu', 'man']
['african', 'queen', 'marriag']
['zulu', 'king']
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 3. - Calcul des poids TF-IDF des termes dans les documents

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours. Utiliser le logarithme en base 10.


Exercice : modifier la fonction `boolean_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [ ]:
# Exercice 3. calcul des scores tf-idf
def compute_tfidf(self):
    """
    Calcule les scores tf-idf pour tous les mots de tous les documents et les stocke dans self.tfidf.

    Dans le code ci-dessous, les scores tf-idf sont tous nuls.
    """
    print("Calculating tf-idf...")

    self.tfidf = defaultdict(Counter)
    tf = defaultdict(Counter)

    i=0
    for i in range(len(self.docs)):
      for term in self.docs[i]:
        tf[i][term] += 1

    N = len(self.docs)  # N = nombre de documents
    for word in self.vocab:
        for i in range(N):
          idf = math.log10(N/len(self.get_posting(word)))
          try:
              self.tfidf[i][word] = (1+math.log10(tf[i][word])) * idf
          except ValueError:
              self.tfidf[i][word] = 0.

# Ne pas modifier les lignes ci-dessous
IRSystem.compute_tfidf = compute_tfidf
ir_system.compute_tfidf()
run_tests(ir_system, part=2)

Calculating tf-idf...
===== Running tests =====
TF-IDF Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000


### Exercice 4. - Calcul de la similarité cosinus.

Dans ce troisième exercice, l'objectif est de pré-calculer les poids TF-IDF pour chaque terme dans chaque document. Pour ce faire, appliquer la formule vue en cours en considérant les éléments suivants :
- Ne considérer que la mesure TF pour calculer le poids des termes de la requête (on utilise pas IDF).
- utiliser le logarithme en base 10.

Exercice : modifier la fonction `rank_retrieve` pour implémenter la recherche booléenne.

Le résultat ci-dessous indique que vous avez réussi.
```
===== Running tests =====
Cosine Similarity Test
    Score: 3 Feedback: 5/5 Correct. Accuracy: 1.000000
```

In [ ]:
# Exercice 4. Similarité cosinus
def rank_retrieve(self, query):
    """
    A partir d'une requête (une liste de mots), retourne une liste de documents (classés par docID) et de scores pour la requête en appliquant la simalirité cosinus.

    Dans l'exemple ci-dessous. C'est la mesure de Jaccard qui est utilisée.
    """
    scores = [0.0 for _ in range(len(self.docs))]

    query_set = set(query)

    for d in range(len(self.docs)):
        doc_set = set(self.docs[d])
        intersection = len(query_set & doc_set)
        union = len(query_set | doc_set)

        ## Code perdu actualisation bug

    # Tri des scores
    ranking = [idx for idx, sim in sorted(enumerate(scores),
                                        key=lambda pair: pair[1], reverse=True)]
    results = []
    for i in range(10):
        results.append((ranking[i], scores[ranking[i]]))
    return results

# Ne pas modifier les lignes ci-dessous
IRSystem.rank_retrieve = rank_retrieve
run_tests(ir_system, part=3)